In [4]:
import pandas as pd
df= pd.read_csv('OR_df.csv')
df.head(2)

,No,Title,Inventors,Publication number,Earliest priority,IPC,CPC,Publication date,Earliest publication,Family number,...,applicant country,first applicant,second applicant,first applicant type,second applicant type,Patent Age,patent cost,Patent Value,Market Rate,Mean Value
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \nDESHMANE ATUL [US] \nME...,US2006250902A1,2005-05-05,['H04B1/20'],"['B60K6/20 (KR)', 'B60K6/28 (KR)', 'B60K6/30 (...",2006-11-09,2006-11-09,37397092,...,US,AFS TRINITY POWER CORP [US],NaN,Company - General,Unknown,20,1500.0,1500.0,5190.48583,5190.48583
1,2,Electric vehicle wireless charging system thro...,SON JEONG KI [KR],KR20230163874A,2022-05-24,"['B60L53/12', 'B60L53/38', 'B60L53/54', 'B60L5...","['B60L53/12 (KR)', 'B60L53/38 (KR)', 'B60L53/5...",2023-12-01,2023-12-01,89124565,...,KR,SON JEONG KI [KR],NaN,Individual Inventor,Unknown,3,4000.0,4000.0,5190.48583,5190.48583


In [5]:
or_df = pd.read_csv('OR_df_exp.csv')

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [7]:
df.columns

Index(['No', 'Title', 'Inventors', 'Publication number', 'Earliest priority',
       'IPC', 'CPC', 'Publication date', 'Earliest publication',
       'Family number', 'first publication date', 'second publication date',
       'first publication number', 'second publication number',
       'first publication country', 'second publication country',
       'first filing year', 'earliest priority year', 'applicant country',
       'first applicant', 'second applicant', 'first applicant type',
       'second applicant type', 'Patent Age', 'patent cost', 'Patent Value',
       'Market Rate', 'Mean Value'],
      dtype='object')

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

class EspacenetScraper:
    def __init__(self, headless=True):
        """Initialize the scraper with configurable options."""
        options = uc.ChromeOptions()
        if headless:
            options.add_argument('--headless')  # Run in headless mode

        options.add_argument('--disable-blink-features=AutomationControlled')
        self.driver = uc.Chrome(options=options)
        self.driver.set_page_load_timeout(30)
        self.driver.set_window_size(1600, 1300)

    def add_random_delay(self, min_seconds=1, max_seconds=3):
        """Add a random delay to mimic human behavior."""
        time.sleep(random.uniform(min_seconds, max_seconds))

    def get_page_html(self, url, retries=3):
        """
        Navigate to the given URL and return the page HTML.
        Retry the operation if a timeout occurs.

        Args:
            url (str): The URL to navigate to.
            retries (int): Number of retry attempts.

        Returns:
            str: The page HTML, or None if all retries fail.
        """
        for attempt in range(retries):
            try:
                print(f"Navigating to: {url} (Attempt {attempt + 1})")
                self.driver.get(url)
                WebDriverWait(self.driver, 30).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )

                # Add a random delay to mimic human behavior
                self.add_random_delay(3, 5)

                # Return the page HTML
                return self.driver.page_source

            except TimeoutException:
                print(f"Timed out waiting for the page to load. Retrying ({attempt + 1}/{retries})...")
                if attempt == retries - 1:
                    print("Max retries reached. Unable to load the page.")
                    return None
            except Exception as e:
                print(f"An error occurred: {e}")
                return None

    def click_citations_tab_and_get_html(self, retries=3):
        """
        Click the "Citations" tab and return the HTML content of the table's tbody.
        If no citations are found (error message is present), return None.

        Args:
            retries (int): Number of retry attempts.

        Returns:
            str or None: The HTML content of the table's tbody, or None if no citations are found.
        """
        for attempt in range(retries):
            try:
                # Wait for the "Citations" tab to be clickable
                citations_tab = WebDriverWait(self.driver, 30).until(
                    EC.element_to_be_clickable((By.XPATH, "//li[@role='tab']//span[contains(text(), 'Citations')]"))
                )
                print("Citations tab found. Clicking...")
                citations_tab.click()
                self.add_random_delay(2, 4)

                # Wait for the wrapper to load
                WebDriverWait(self.driver, 30).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.wrapper--m6N288zA"))
                )
                print("Wrapper found.")

                # Check if the table exists inside the wrapper
                table = self.driver.find_elements(By.CSS_SELECTOR, "div.wrapper--m6N288zA table.table__table--8dvWEm4j")
                if table:
                    print("Table found inside the wrapper.")
                    # Wait for the table's tbody to load
                    WebDriverWait(self.driver, 30).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "table.table__table--8dvWEm4j tbody.table__body--wKJVkwCZ"))
                    )
                    print("Table body found.")

                    # Add a delay to ensure the table is fully rendered
                    self.add_random_delay(2, 4)

                    # Extract the HTML content of the table's tbody
                    tbody = self.driver.find_element(By.CSS_SELECTOR, "table.table__table--8dvWEm4j tbody.table__body--wKJVkwCZ")
                    return tbody.get_attribute("outerHTML")
                else:
                    # Check if the error message exists
                    error_message = self.driver.find_elements(By.CSS_SELECTOR, "div.wrapper--m6N288zA div.error__container--85Brg4Ek")
                    if error_message:
                        print("No citations found for this patent.")
                        return None  # Return None if no citations are found
                    else:
                        print("Neither table nor error message found. Retrying...")
                        continue  # Retry the loop

            except TimeoutException:
                print(f"Timed out waiting for the 'Citations' tab to load. Retrying ({attempt + 1}/{retries})...")
                if attempt == retries - 1:
                    print("Max retries reached. Unable to load the 'Citations' tab.")
                    return None
            except Exception as e:
                print(f"An error occurred while clicking the 'Citations' tab: {e}")
                return None

    def parse_tbody_to_dataframe(self, tbody_html):
        """
        Parse the tbody HTML and convert it into a pandas DataFrame.

        Args:
            tbody_html (str): The HTML content of the tbody.

        Returns:
            pd.DataFrame: A DataFrame containing the extracted data.
        """
        soup = BeautifulSoup(tbody_html, 'html.parser')
        rows = soup.find_all('tr', class_='table__row--qDMbhLBz')

        data = []
        for row in rows:
            cells = row.find_all('td', class_='table__cell--9cdFfes6')
            row_data = []
            for cell in cells:
                # Extract text from the span if it exists, otherwise extract text from the cell
                span = cell.find('span')
                if span:
                    row_data.append(span.get_text(strip=True))
                else:
                    row_data.append(cell.get_text(strip=True))
            data.append(row_data)

        # Define column names based on the table structure
        columns = [
            "CitationOrigin", "Publication", "Title", "Earliest PriorityDate",
            "PublicationDate", "Applicants", "IPC", "CPC"
        ]

        # Create a DataFrame
        df = pd.DataFrame(data, columns=columns)
        return df

    def close(self):
        """Close the browser when done."""
        if self.driver:
            self.driver.quit()


if __name__ == '__main__':
    # Load your input DataFrame - THIS WAS MISSING IN ORIGINAL CODE
    # Assuming you have a CSV file with your patents data
    df = pd.read_csv('OR_df_exp.csv')  # Replace with your actual file path
    
    # Make sure the DataFrame has the required columns
    required_columns = ['Family number', 'first publication number']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Input DataFrame must contain these columns: {required_columns}")
    
    # Initialize the scraper
    scraper = EspacenetScraper(headless=False)  # Set headless to False to see the browser in action

    # Create a new empty DataFrame to store the results
    # Initialize with the same columns as the input DataFrame plus the new 'citations IPC' column
    result_df = pd.DataFrame(columns=df.columns.tolist() + ['citations IPC'])

    # Counter to keep track of non-empty lists in the 'citations IPC' column
    non_empty_count = 0

    try:
        # Iterate over each row in the original DataFrame
        for index, row in df.iterrows():
            # Stop if we have at least 30 rows with non-empty citations
            if non_empty_count >= 30:
                break

            # Construct the URL for the current patent
            url = f"https://worldwide.espacenet.com/patent/search/family/{row['Family number']}/publication/{row['first publication number']}?q=pn%3D{row['first publication number']}"
            print(f"Processing patent: {row['first publication number']}")

            # Get the page HTML
            html = scraper.get_page_html(url, retries=3)
            if html:
                print("Page HTML retrieved successfully.")

                # Click the "Citations" tab and get its HTML content
                tbody_html = scraper.click_citations_tab_and_get_html(retries=3)
                if tbody_html:
                    print(f"Citations tab tbody HTML retrieved successfully for {row['first publication number']}.")

                    # Parse the tbody HTML and convert it into a DataFrame
                    citations_df = scraper.parse_tbody_to_dataframe(tbody_html)
                    if citations_df is not None and not citations_df.empty:  # Check if citations were found
                        print(f"DataFrame created successfully for {row['first publication number']}.")

                        # Extract the IPC classes from the citations DataFrame
                        ipc_list = citations_df["IPC"].tolist()

                        # Create a new row as a dictionary
                        new_row = row.to_dict()
                        new_row['citations IPC'] = ipc_list

                        # Append the row to the result DataFrame - FIX FOR THE ERROR
                        # Create a temporary DataFrame with the new row and ensure index is unique
                        temp_df = pd.DataFrame([new_row])
                        result_df = pd.concat([result_df, temp_df], ignore_index=True)

                        # Increment the non-empty count
                        non_empty_count += 1
                        print(f"Added patent {row['first publication number']} with {len(ipc_list)} citation IPCs. Total with citations: {non_empty_count}")
                    else:
                        print(f"No citations found for {row['first publication number']}.")
                else:
                    print(f"No citations found for {row['first publication number']}.")

    finally:
        # Close the browser
        scraper.close()
        print("Scraper closed.")

        # Display the resulting DataFrame
        if not result_df.empty:
            print(result_df[['first publication number', 'citations IPC']])


            # Save the resulting DataFrame to a CSV file
            result_df.to_csv("patents_with_citations_ipc.csv", index=False, encoding="utf-8")
            print("DataFrame saved to 'patents_with_citations_ipc.csv'.")
        else:
            print("No results collected. Result DataFrame is empty.")

Processing patent: US2006250902A1
Navigating to: https://worldwide.espacenet.com/patent/search/family/37397092/publication/US2006250902A1?q=pn%3DUS2006250902A1 (Attempt 1)
Page HTML retrieved successfully.
Timed out waiting for the 'Citations' tab to load. Retrying (1/3)...
Citations tab found. Clicking...
Wrapper found.
Table found inside the wrapper.
Table body found.
Citations tab tbody HTML retrieved successfully for US2006250902A1.
DataFrame created successfully for US2006250902A1.
Added patent US2006250902A1 with 15 citation IPCs. Total with citations: 1
Processing patent: KR20230163874A
Navigating to: https://worldwide.espacenet.com/patent/search/family/89124565/publication/KR20230163874A?q=pn%3DKR20230163874A (Attempt 1)


C:\Users\tasni\AppData\Local\Temp\ipykernel_27208\1681725418.py:222: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, temp_df], ignore_index=True)


Page HTML retrieved successfully.
Citations tab found. Clicking...
Wrapper found.
Table found inside the wrapper.
Table body found.
Citations tab tbody HTML retrieved successfully for KR20230163874A.
DataFrame created successfully for KR20230163874A.
Added patent KR20230163874A with 4 citation IPCs. Total with citations: 2
Processing patent: KR102511391B1
Navigating to: https://worldwide.espacenet.com/patent/search/family/85796535/publication/KR102511391B1?q=pn%3DKR102511391B1 (Attempt 1)
Page HTML retrieved successfully.
Citations tab found. Clicking...
Wrapper found.
Table found inside the wrapper.
Table body found.
Citations tab tbody HTML retrieved successfully for KR102511391B1.
DataFrame created successfully for KR102511391B1.
Added patent KR102511391B1 with 5 citation IPCs. Total with citations: 3
Processing patent: WO2011019133A2
Navigating to: https://worldwide.espacenet.com/patent/search/family/42183715/publication/WO2011019133A2?q=pn%3DWO2011019133A2 (Attempt 1)
Page HTML re

In [9]:
result_df[ 'citations IPC'].iloc[2]

['B60L53/10, B60L53/24, B60L53/66, H02H5/04, H02H7/18, H02M3/00, H02M7/44',
 'B60L53/10, B60L53/24, B60L53/66, H02H5/04, H02H7/18, H02M3/00, H02M7/44',
 'F17C11/00, H01M8/04',
 'C25B1/04, H01M10/44, H01M10/48, H01M8/00, H01M8/04, H02J15/00, H02J3/32, H02J3/38',
 'B60K15/03, B60L3/00, B60L50/72, B60L53/30, B60P3/22']

In [10]:
result_df.head()

,No,Title,Inventors,Publication number,Earliest priority,IPC,CPC,Publication date,Earliest publication,Family number,...,first applicant,second applicant,first applicant type,second applicant type,Patent Age,patent cost,Patent Value,Market Rate,Mean Value,citations IPC
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \nDESHMANE ATUL [US] \nME...,US2006250902A1,2005-05-05,['H04B1/20'],"['B60K6/20 (KR)', 'B60K6/28 (KR)', 'B60K6/30 (...",2006-11-09,2006-11-09,37397092,...,AFS TRINITY POWER CORP [US],NaN,Company - General,Unknown,20,1500.0,1500.0,5190.48583,5190.48583,"[B60K1/00, B60K6/00, B60K6/28, B60K6/52, B60L1..."
1,2,Electric vehicle wireless charging system thro...,SON JEONG KI [KR],KR20230163874A,2022-05-24,"['B60L53/12', 'B60L53/38', 'B60L53/54', 'B60L5...","['B60L53/12 (KR)', 'B60L53/38 (KR)', 'B60L53/5...",2023-12-01,2023-12-01,89124565,...,SON JEONG KI [KR],NaN,Individual Inventor,Unknown,3,4000.0,4000.0,5190.48583,5190.48583,"[B60L11/18, B60M7/00, H01M10/44, H02J3/32, H02..."
2,3,Electric vehicle charging system using hydroge...,HONG SEONG HO,KR102511391B1,2022-09-20,"['B60L3/00', 'B60L53/16', 'B60L53/51', 'B60L53...",['Y02T10/70 (EP)'],2023-03-20,2023-03-20,85796535,...,HOGREENAIR CO LTD [KR],NaN,Company - Limited,Unknown,3,4000.0,4000.0,5190.48583,5190.48583,"[B60L53/10, B60L53/24, B60L53/66, H02H5/04, H0..."
3,5,ESS SYSTEM FOR CHARGING FUEL CELL ELECTRIC VEH...,MUN TAEEUN [KR] \nCHAE HO BYUNG [KR] \nSHIN JO...,US2022140365A1,2020-11-02,"['H01M16/00', 'H01M8/04828', 'H02J7/34']","['B60K15/03006 (KR)', 'B60L53/20 (KR)', 'B60L5...",2022-05-05,2021-03-17,75243546,...,SIGNET EV INC [KR],SIGNET ENERGY [KR],Company - Incorporated/Corporation,Energy Company,5,7000.0,7000.0,5190.48583,5190.48583,"[B60L50/75, B60L53/30, B60L53/53, B60L53/54, F..."
4,6,Method for quickly charging electric vehicle l...,CHENG HUANGREN \nHAN YIXI \nLI QINGBAO \nZHAO XI,CN108550935A,2018-04-19,['H01M10/44'],"['H01M10/445 (CN)', 'Y02E60/10 (EP)']",2018-09-18,2018-09-18,63515557,...,TIANJIN YADI IND CO LTD,NaN,Company - Limited,Unknown,7,2000.0,2000.0,5190.48583,5190.48583,"[H01M10/44, H01M10/44, H02J7/00, H01M10/44, H0..."


In [11]:
df.head()

,No,Title,Inventors,Publication number,Earliest priority,IPC,CPC,Publication date,Earliest publication,Family number,...,applicant country,first applicant,second applicant,first applicant type,second applicant type,Patent Age,patent cost,Patent Value,Market Rate,Mean Value
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \nDESHMANE ATUL [US] \nME...,US2006250902A1,2005-05-05,['H04B1/20'],"['B60K6/20 (KR)', 'B60K6/28 (KR)', 'B60K6/30 (...",2006-11-09,2006-11-09,37397092,...,US,AFS TRINITY POWER CORP [US],NaN,Company - General,Unknown,20,1500.0,1500.0,5190.48583,5190.48583
1,2,Electric vehicle wireless charging system thro...,SON JEONG KI [KR],KR20230163874A,2022-05-24,"['B60L53/12', 'B60L53/38', 'B60L53/54', 'B60L5...","['B60L53/12 (KR)', 'B60L53/38 (KR)', 'B60L53/5...",2023-12-01,2023-12-01,89124565,...,KR,SON JEONG KI [KR],NaN,Individual Inventor,Unknown,3,4000.0,4000.0,5190.48583,5190.48583
2,3,Electric vehicle charging system using hydroge...,HONG SEONG HO,KR102511391B1,2022-09-20,"['B60L3/00', 'B60L53/16', 'B60L53/51', 'B60L53...",['Y02T10/70 (EP)'],2023-03-20,2023-03-20,85796535,...,KR,HOGREENAIR CO LTD [KR],NaN,Company - Limited,Unknown,3,4000.0,4000.0,5190.48583,5190.48583
3,4,BATTERY-CHARGING SYSTEM FOR AN ELECTRIC VEHICLE,CHUNG YON JONG [KR],WO2011019133A2 \nWO2011019133A3,2009-08-13,"['B60L11/18', 'B60W10/24', 'H02J7/00']","['B60K6/46 (EP)', 'B60L15/2045 (EP)', 'B60L58/...",2011-02-17 \n2011-04-07,2010-03-25,42183715,...,KR,CHUNG YON JONG [KR],NaN,Individual Inventor,Unknown,16,5300.0,5300.0,5190.48583,5190.48583
4,5,ESS SYSTEM FOR CHARGING FUEL CELL ELECTRIC VEH...,MUN TAEEUN [KR] \nCHAE HO BYUNG [KR] \nSHIN JO...,US2022140365A1,2020-11-02,"['H01M16/00', 'H01M8/04828', 'H02J7/34']","['B60K15/03006 (KR)', 'B60L53/20 (KR)', 'B60L5...",2022-05-05,2021-03-17,75243546,...,KR,SIGNET EV INC [KR],SIGNET ENERGY [KR],Company - Incorporated/Corporation,Energy Company,5,7000.0,7000.0,5190.48583,5190.48583


In [12]:
def process_citations(citations):
    # Initialize a set to store unique citations
    processed = set()
    
    # Iterate over each citation string
    for citation in citations:
        # Split the citation string by commas
        parts = citation.split(',')
        # Add each part to the set (strip whitespace)
        for part in parts:
            # Split by '/' and take the first part, then strip whitespace
            processed.add(part.split('/')[0].strip())

    unique_citaions = list(processed)
    unique_citaions.sort()
    
    # Convert the set back to a list
    return unique_citaions

In [13]:
result_df['citations IPC'] = result_df['citations IPC'].apply(process_citations)
result_df.head()

,No,Title,Inventors,Publication number,Earliest priority,IPC,CPC,Publication date,Earliest publication,Family number,...,first applicant,second applicant,first applicant type,second applicant type,Patent Age,patent cost,Patent Value,Market Rate,Mean Value,citations IPC
0,1,Plug-in hybrid vehicle with fast energy storage,BENDER DONALD A [US] \nDESHMANE ATUL [US] \nME...,US2006250902A1,2005-05-05,['H04B1/20'],"['B60K6/20 (KR)', 'B60K6/28 (KR)', 'B60K6/30 (...",2006-11-09,2006-11-09,37397092,...,AFS TRINITY POWER CORP [US],NaN,Company - General,Unknown,20,1500.0,1500.0,5190.48583,5190.48583,"[B06L1, B60K1, B60K17, B60K21, B60K23, B60K25,..."
1,2,Electric vehicle wireless charging system thro...,SON JEONG KI [KR],KR20230163874A,2022-05-24,"['B60L53/12', 'B60L53/38', 'B60L53/54', 'B60L5...","['B60L53/12 (KR)', 'B60L53/38 (KR)', 'B60L53/5...",2023-12-01,2023-12-01,89124565,...,SON JEONG KI [KR],NaN,Individual Inventor,Unknown,3,4000.0,4000.0,5190.48583,5190.48583,"[B60L11, B60L53, B60M7, F16H7, G06Q50, H01M10,..."
2,3,Electric vehicle charging system using hydroge...,HONG SEONG HO,KR102511391B1,2022-09-20,"['B60L3/00', 'B60L53/16', 'B60L53/51', 'B60L53...",['Y02T10/70 (EP)'],2023-03-20,2023-03-20,85796535,...,HOGREENAIR CO LTD [KR],NaN,Company - Limited,Unknown,3,4000.0,4000.0,5190.48583,5190.48583,"[B60K15, B60L3, B60L50, B60L53, B60P3, C25B1, ..."
3,5,ESS SYSTEM FOR CHARGING FUEL CELL ELECTRIC VEH...,MUN TAEEUN [KR] \nCHAE HO BYUNG [KR] \nSHIN JO...,US2022140365A1,2020-11-02,"['H01M16/00', 'H01M8/04828', 'H02J7/34']","['B60K15/03006 (KR)', 'B60L53/20 (KR)', 'B60L5...",2022-05-05,2021-03-17,75243546,...,SIGNET EV INC [KR],SIGNET ENERGY [KR],Company - Incorporated/Corporation,Energy Company,5,7000.0,7000.0,5190.48583,5190.48583,"[B60L50, B60L53, B60S5, B64B1, B64C39, B64D27,..."
4,6,Method for quickly charging electric vehicle l...,CHENG HUANGREN \nHAN YIXI \nLI QINGBAO \nZHAO XI,CN108550935A,2018-04-19,['H01M10/44'],"['H01M10/445 (CN)', 'Y02E60/10 (EP)']",2018-09-18,2018-09-18,63515557,...,TIANJIN YADI IND CO LTD,NaN,Company - Limited,Unknown,7,2000.0,2000.0,5190.48583,5190.48583,"[H01M10, H02J7]"


In [18]:
def process_ipc_codes(ipc_list):
    """
    Process a list of IPC codes with country codes in parentheses.
    
    Args:
        ipc_list (list or str): A list of IPC codes in the format "CODE (COUNTRY)" or a string
                               representation of such a list.
                               
    Returns:
        list: The original list without any modifications.
    
    Example:
        >>> input_list = ["H01M8/04201 (CN)", "H01M8/0618 (CN)", "H01M8/0625 (CN)", 
                          "Y02E60/50 (EP)", "Y02T90/40 (EP)"]
        >>> process_ipc_codes(input_list)
        ['H01M8/04201 (CN)', 'H01M8/0618 (CN)', 'H01M8/0625 (CN)', 'Y02E60/50 (EP)', 'Y02T90/40 (EP)']
    """
    # If the input is a string (as might happen if extracted from text),
    # convert it to a list by parsing the string
    if isinstance(ipc_list, str):
        # Check if it's a string representation of a list (starts with [ and ends with ])
        if ipc_list.strip().startswith('[') and ipc_list.strip().endswith(']'):
            # Remove the brackets and split by commas
            # Use eval carefully - only if you're sure the input is safe
            try:
                ipc_list = eval(ipc_list)
            except:
                # If eval fails, do a simple string split and clean up
                content = ipc_list.strip()[1:-1]  # Remove brackets
                ipc_list = [item.strip().strip("'\"") for item in content.split(',')]
    
    # Ensure the result is a list
    if not isinstance(ipc_list, list):
        ipc_list = [ipc_list]
    
    # Return the original list (as per your request)
    return ipc_list

# Example usage:
if __name__ == "__main__":
    input_list = ["H01M8/04201 (CN)", "H01M8/0618 (CN)", "H01M8/0625 (CN)", 
                  "Y02E60/50 (EP)", "Y02T90/40 (EP)"]
    result = process_ipc_codes(input_list)
    print(result)
    
    # Test with a string input
    string_input = "['H01M8/04201 (CN)', 'H01M8/0618 (CN)', 'H01M8/0625 (CN)', 'Y02E60/50 (EP)', 'Y02T90/40 (EP)']"
    result2 = process_ipc_codes(string_input)
    print(result2)

['H01M8/04201 (CN)', 'H01M8/0618 (CN)', 'H01M8/0625 (CN)', 'Y02E60/50 (EP)', 'Y02T90/40 (EP)']
['H01M8/04201 (CN)', 'H01M8/0618 (CN)', 'H01M8/0625 (CN)', 'Y02E60/50 (EP)', 'Y02T90/40 (EP)']


In [15]:
result = process_citations(result)

In [19]:
result

['H01M8/04201 (CN)',
 'H01M8/0618 (CN)',
 'H01M8/0625 (CN)',
 'Y02E60/50 (EP)',
 'Y02T90/40 (EP)']

In [20]:
result_df['IPC'] = result_df['IPC'].apply(process_ipc_codes)

In [22]:
result_df['IPC'] = result_df['IPC'].apply(process_citations)

In [23]:
result_df['CPC'] = result_df['CPC'].apply(process_ipc_codes)
result_df['CPC'] = result_df['CPC'].apply(process_citations)

In [24]:
result_df['IPC'].head(2)

0                     [H04B1]
1    [B60L53, G06Q20, G06Q50]
Name: IPC, dtype: object

In [25]:
citations_IPC_df= result_df[['first publication number','IPC','citations IPC']]

In [26]:
citations_IPC_df['IPC count']= citations_IPC_df['IPC'].apply(len)
citations_IPC_df['citations IPC count']= citations_IPC_df['citations IPC'].apply(len)

C:\Users\tasni\AppData\Local\Temp\ipykernel_27208\4076942067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citations_IPC_df['IPC count']= citations_IPC_df['IPC'].apply(len)
C:\Users\tasni\AppData\Local\Temp\ipykernel_27208\4076942067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citations_IPC_df['citations IPC count']= citations_IPC_df['citations IPC'].apply(len)


check empty rows

In [27]:
citations_IPC_df['%citaions'] = citations_IPC_df['IPC count'] / citations_IPC_df['citations IPC count']


C:\Users\tasni\AppData\Local\Temp\ipykernel_27208\65443135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citations_IPC_df['%citaions'] = citations_IPC_df['IPC count'] / citations_IPC_df['citations IPC count']


In [33]:
citations_IPC_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   first publication number  24 non-null     object 
 1   IPC                       24 non-null     object 
 2   citations IPC             24 non-null     object 
 3   IPC count                 24 non-null     int64  
 4   citations IPC count       24 non-null     int64  
 5   %citaions                 24 non-null     float64
 6   %S²                       24 non-null     float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.4+ KB


In [37]:
citations_IPC_df = citations_IPC_df[citations_IPC_df['IPC count'] < citations_IPC_df['citations IPC count']]

In [38]:
citations_IPC_df



,first publication number,IPC,citations IPC,IPC count,citations IPC count,%citaions,%S²
0,US2006250902A1,[H04B1],"[B06L1, B60K1, B60K17, B60K21, B60K23, B60K25,...",1,40,0.025000,0.000625
1,KR20230163874A,"[B60L53, G06Q20, G06Q50]","[B60L11, B60L53, B60M7, F16H7, G06Q50, H01M10,...",3,8,0.375000,0.140625
2,KR102511391B1,"[B60L3, B60L53, B60L58, H01M8, H05K7]","[B60K15, B60L3, B60L50, B60L53, B60P3, C25B1, ...",5,15,0.333333,0.111111
3,US2022140365A1,"[H01M16, H01M8, H02J7]","[B60L50, B60L53, B60S5, B64B1, B64C39, B64D27,...",3,22,0.136364,0.018595
4,CN108550935A,[H01M10],"[H01M10, H02J7]",1,2,0.500000,0.250000
5,CN109263516A,"[B60L58, H01M8]","[, B60L1, B60L11, B60L15, B60L50, B60L58, H01M...",2,9,0.222222,0.049383
6,US2013093378A1,[H02J7],"[B60L11, B60L8, B63C7, B63H11, B63H13, B63H21,...",1,27,0.037037,0.001372
7,CN112606713A,"[B60L50, B60L58, H02J7, H02P27]","[B60K1, B60L11, B60L50, B60L58, B60L7, H01M10,...",4,10,0.400000,0.160000
8,WO2023128748A1,"[B60K6, B60L15, B60L53, B60L58]","[B01J19, B01J7, B60K1, B60L1, B60L11, B60L58, ...",4,15,0.266667,0.071111
10,CN108674229A,"[B60L50, B60L53, B60S5, H02J7]","[B60C19, B60K15, B60L11, B60S5, B65B1, B65B3, ...",4,17,0.235294,0.055363


**originality rate**

In [39]:
citations_IPC_df['%S²'] = citations_IPC_df['%citaions']**2
mean_S_square = citations_IPC_df['%S²'].sum()
num_rows = len(citations_IPC_df)
original_rate = 1- mean_S_square/num_rows
print('num rows = ' , num_rows)
print('OR = ' ,' {:.2f}%'.format(original_rate) )

num rows =  21
OR =   0.89%


C:\Users\tasni\AppData\Local\Temp\ipykernel_27208\2216545038.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  citations_IPC_df['%S²'] = citations_IPC_df['%citaions']**2


In [42]:
df['citations IPC'].head()

0    [B60K1/00, B60K6/00, B60K6/28, B60K6/52, B60L1...
1    [B60L11/18, B60M7/00, H01M10/44, H02J3/32, H02...
2    [B60L53/10, B60L53/24, B60L53/66, H02H5/04, H0...
3                                                   []
4    [B60L50/75, B60L53/30, B60L53/53, B60L53/54, F...
Name: citations IPC, dtype: object

In [43]:
df['citations IPC'] = df['citations IPC'].apply(process_citations)

In [44]:
df['citations IPC']

0     [B06L1, B60K1, B60K17, B60K21, B60K23, B60K25,...
1     [B60L11, B60L53, B60M7, F16H7, G06Q50, H01M10,...
2     [B60K15, B60L3, B60L50, B60L53, B60P3, C25B1, ...
3                                                    []
4     [B60L50, B60L53, B60S5, B64B1, B64C39, B64D27,...
5                                       [H01M10, H02J7]
6     [, B60L1, B60L11, B60L15, B60L50, B60L58, H01M...
7     [B60L11, B60L8, B63C7, B63H11, B63H13, B63H21,...
8                                                    []
9                                                    []
10    [B60K1, B60L11, B60L50, B60L58, B60L7, H01M10,...
11    [B01J19, B01J7, B60K1, B60L1, B60L11, B60L58, ...
12                                                   []
13                                                   []
14                              [B60L11, B60L15, B60L7]
15    [B60C19, B60K15, B60L11, B60S5, B65B1, B65B3, ...
16                                                   []
17                                              

In [40]:
# Input data
citations = [
    'B60L53/10, B60L53/24, B60L53/66, H02H5/04, H02H7/18, H02M3/00, H02M7/44',
    'B60L53/10, B60L53/24, B60L53/66, H02H5/04, H02H7/18, H02M3/00, H02M7/44',
    'F17C11/00, H01M8/04',
    'C25B1/04, H01M10/44, H01M10/48, H01M8/00, H01M8/04, H02J15/00, H02J3/32, H02J3/38',
    'B60K15/03, B60L3/00, B60L50/72, B60L53/30, B60P3/22'
]

# Process the citations
processed_citations = process_citations(citations)

# Display the results
print("Processed Citations:")
print(processed_citations)

Processed Citations:
['B60K15', 'B60L3', 'B60L50', 'B60L53', 'B60P3', 'C25B1', 'F17C11', 'H01M10', 'H01M8', 'H02H5', 'H02H7', 'H02J15', 'H02J3', 'H02M3', 'H02M7']


In [41]:
len(processed_citations)

15

In [45]:
df['citations IPC'].iloc[2]

['B60K15',
 'B60L3',
 'B60L50',
 'B60L53',
 'B60P3',
 'C25B1',
 'F17C11',
 'H01M10',
 'H01M8',
 'H02H5',
 'H02H7',
 'H02J15',
 'H02J3',
 'H02M3',
 'H02M7']